<a href="https://colab.research.google.com/github/ssrakuen/skripsi-notebook/blob/main/Resnet_Architectures_Comparison_with_Sleep_Pose_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install necessary libraries
!pip install torch torchvision tensorboard scikit-learn tqdm

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from torch.utils.tensorboard import SummaryWriter
import time
import os
import numpy as np
from tqdm import tqdm

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

# Set Device, Dataset, and Train Settings

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
data_dir = '/content/drive/MyDrive/Machine_Learning/Datasets/sleep_pose_dataset/A1A2'

train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define ResNet architectures to test
resnet_versions = {
    # 'resnet18': models.resnet18(weights=models.ResNet18_Weights.DEFAULT),
    # 'resnet34': models.resnet34(weights=models.ResNet34_Weights.DEFAULT),
    # 'resnet50': models.resnet50(weights=models.ResNet50_Weights.DEFAULT),
    'resnet101': models.resnet101(weights=models.ResNet101_Weights.DEFAULT),
    'resnet152': models.resnet152(weights=models.ResNet152_Weights.DEFAULT)
}

# Modify the last fully connected layer to match the number of classes (3 in this case)
for version, model in resnet_versions.items():
    model.fc = nn.Linear(model.fc.in_features, 3)
    resnet_versions[version] = model.to(device)

# Define hyperparameters and parameter grid for GridSearch
param_grid = {
    'lr': [0.001, 0.0001],
    'batch_size': [16, 32, 64]
}

# Initialize TensorBoard writer
writer = SummaryWriter()

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth
100%|██████████| 171M/171M [00:02<00:00, 87.1MB/s]
Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:02<00:00, 84.2MB/s]


# Train and Eval

In [ ]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    best_model_wts = None
    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        writer.add_scalar('Loss/train', epoch_loss, epoch)
        writer.add_scalar('Accuracy/train', epoch_acc, epoch)

        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f'Validation Epoch {epoch+1}/{num_epochs}'):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)

        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()

        print(f'Epoch {epoch}/{num_epochs-1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    if best_model_wts:
        model.load_state_dict(best_model_wts)

    return model

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Evaluating'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    return accuracy, precision, recall, f1

# Main function to run the training and evaluation
def main():
    for version, model in resnet_versions.items():
        criterion = nn.CrossEntropyLoss()

        for lr in param_grid['lr']:
            for batch_size in param_grid['batch_size']:
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                optimizer = optim.Adam(model.parameters(), lr=lr)

                print(f'Training {version} with lr={lr} and batch_size={batch_size}')
                model = train_model(model, train_loader, val_loader, criterion, optimizer)

                accuracy, precision, recall, f1 = evaluate_model(model, test_loader)

                # Log metrics to TensorBoard
                writer.add_hparams({'lr': lr, 'batch_size': batch_size, 'model': version},
                                   {'hparam/accuracy': accuracy, 'hparam/precision': precision,
                                    'hparam/recall': recall, 'hparam/f1': f1})

                # Save the model
                model_save_path = f'/content/drive/MyDrive/Machine_Learning/modelsA1A2/{version}_lr{lr}_batch{batch_size}.pth'
                os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
                torch.save(model.state_dict(), model_save_path)

                # Save statistics
                stats_save_path = f'/content/drive/MyDrive/Machine_Learning/statsA1A2/{version}_lr{lr}_batch{batch_size}_stats.npy'
                os.makedirs(os.path.dirname(stats_save_path), exist_ok=True)
                np.save(stats_save_path, {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1})

    writer.close()

# Run the main function
if __name__ == '__main__':
    main()

Training resnet101 with lr=0.001 and batch_size=16


Validation Epoch 1/25: 100%|██████████| 29/29 [00:04<00:00,  6.38it/s]


Epoch 0/24, Loss: 0.0309, Acc: 0.9933, Val Loss: 0.0026, Val Acc: 1.0000


Validation Epoch 2/25: 100%|██████████| 29/29 [00:04<00:00,  6.30it/s]


Epoch 1/24, Loss: 0.0068, Acc: 0.9983, Val Loss: 0.0071, Val Acc: 0.9978


...

Epoch 23/24, Loss: 0.0000, Acc: 1.0000, Val Loss: 0.0023, Val Acc: 0.9978


Validation Epoch 25/25: 100%|██████████| 8/8 [00:05<00:00,  1.54it/s]


Epoch 24/24, Loss: 0.0000, Acc: 1.0000, Val Loss: 0.0024, Val Acc: 0.9978


Evaluating: 100%|██████████| 18/18 [00:06<00:00,  2.64it/s]


Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
